# Evaluate 
$ f(x, y) = 2x^2 - x^2y^2 + 3 $

$ g_1 = x ⋅ x $ 

$ g_2 = g1 ⋅ 2 $

$ g_3 = y ⋅ y $

$ g_4 = g1 ⋅ g3 $

$ g_5 = g2 - g4 $

$ g_6 = g5 + 3 $

$ c_1 = a_1 ⋅ b_1 $

$ c_2 = a_2 ⋅ 2 $

$ c_3 = a_3 ⋅ b_3 $

$ c_4 = a_4 ⋅ b_4 $

$ c_5 = a_5 - b_5 $

$ c_6 = a_6 + 3 $

$ x = a_1 = b_1 $

$ y = a_3 = b_3 $

$ g_1 = a_2 = c_1 $

$ g_2 = a_5 = c_2 $

$ g_3 = b_4 = c_3 $

$ g_4 = b_5 = c_4 $

$ g_5 = a_6 = c_5 $

$ g_6 = c_6 $

### Gate constrain equation

$ g_i = q_{Li}⋅a_i+q_{Ri}⋅b_i+q_{Mi}⋅(a_i⋅b_i)+q_{Ci}+q_{Oi}⋅c_i=0 $

$ g_1 = 0⋅a_1 + 0⋅b_1 + 1⋅(a_1⋅b_1) + 0 + (-1)⋅c_1 = 0 $

$ g_2 = 2⋅a_2 + 0⋅b_2 + 0⋅(a_2⋅b_2) + 0 + (-1)⋅c_2 = 0 $

$ g_3 = 0⋅a_3 + 0⋅b_3 + 1⋅(a_3⋅b_3) + 0 + (-1)⋅c_3 = 0 $

$ g_4 = 0⋅a_4 + 0⋅b_4 + 0⋅(a_4⋅b_4) + 0 + (-1)⋅c_4 = 0 $

$ g_5 = 1⋅a_5 + (-1)⋅b_5 + 0⋅(a_5⋅b_5) + 0 + (-1)⋅c_5 = 0 $

$ g_6 = 1⋅a_6 + 0⋅b_6 + 0⋅(a_6⋅b_6) + 3 + (-1)⋅c_6 = 0 $

**selector vectors**

$ g_i = [q_{Li}, q_{Ri}, q_{Mi}, q_{Ci}, q_{Oi}] $

$ g_1 = [0, 0, 1, 0, -1] $

$ g_2 = [2, 0, 0, 0, -1] $

$ g_3 = [0, 0, 1, 0, -1] $

$ g_4 = [0, 0, 0, 0, -1] $

$ g_5 = [1, -1, 0, 0, -1] $

$ g_6 = [1, 0, 0, 3, -1] $

**let x = 2 y = 3**


| Vectors| a | b | c |
| --- | --- | --- | --- |
| gate1 | 2 | 2 | 4 |
| gate2 | 4 | 0 | 8 |
| gate3 | 3 | 3 | 9 |
| gate4 | 4 | 9 | 36 |
| gate5 | 8 | 36 | -28 |
| gate6 | -28 | 3 | -25 |

In [23]:
import galois

a = [2, 4, 3, 4, 8, -28]
b = [2, 0, 3, 9, 36, 3]
c = [4, 8, 9, 36, -28, -25]

def to_galois_array(a, GF):
    # normalize to positive values
    a = [x % GF.order for x in a]
    return GF(a)

def to_vanishing_poly(a, GF):
    p = galois.Poly([1, -int(a[0]) % GF.order], field=GF)
    for i in range(1, len(a)):
        p *= galois.Poly([1, -int(a[i]) % GF.order], field=GF)
    return p

p = 73
GF = galois.GF(p)
w = GF(6)

x = GF([w**x for x in range(0, len(a))])

y = to_galois_array(a, GF)
_A = galois.lagrange_poly(x, y)

y = to_galois_array(b, GF)
_B = galois.lagrange_poly(x, y)

y = to_galois_array(c, GF)
_C = galois.lagrange_poly(x, y)

Z = to_vanishing_poly(x, GF)

bA = galois.Poly.Random(degree=1, field=GF)
bB = galois.Poly.Random(degree=1, field=GF)
bC = galois.Poly.Random(degree=1, field=GF)

A = _A + bA*Z
B = _B + bB*Z
C = _C + bC*Z

print("--- Vanishing Polynomial ---")
print("Z = ", Z)

print("\n\n--- Blinding Polynomials ---")
print(f"bA = {bA}")
print(f"bB = {bB}")
print(f"bC = {bC}")

print("\n\n--- Wire Polynomials ---")
print(f"A = {_A}")
print(f"B = {_B}")
print(f"C = {_C}")

print("\n\n--- Blinded Wire Polynomials ---")
print(f"A' = {A}")
print(f"B' = {B}")
print(f"C' = {C}")

--- Vanishing Polynomial ---
Z =  x^6 + 13x^5 + 26x^4 + 7x^3 + 39x^2 + 11x + 49


--- Blinding Polynomials ---
bA = 40x + 39
bB = 13x + 69
bC = 40x + 16


--- Wire Polynomials ---
A = x^5 + 18x^4 + 14x^3 + 45x^2 + 35x + 35
B = 11x^5 + 35x^4 + 23x^3 + 23x^2 + 40x + 16
C = 50x^5 + 8x^4 + 72x^3 + 58x^2 + 72x + 36


--- Blinded Wire Polynomials ---
A' = 40x^7 + 48x^6 + 15x^5 + 71x^4 + 22x^3 + 35x^2 + 15x + 48
B' = 13x^7 + 19x^6 + 5x^5 + 22x^4 + 64x^3 + 10x^2 + 49x + 39
C' = 40x^7 + 25x^6 + 57x^5 + 47x^4 + 65x^3 + 27x^2 + 18x + 17
